In [ ]:
!pip install jupyter-ai

In [ ]:
pip install python-dotenv

In [ ]:

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


domain = os.environ.get('DOMAIN')
admin_email = os.environ.get('ADMIN_EMAIL')
root_url = os.environ.get('ROOT_URL')
token=os.environ.get('OPENAI_API_KEY')

print(f"Domain: {domain}")
print(f"Admin Email: {admin_email}")
print(f"Root URL: {root_url}")
#print(f"Token: {token}")
#api.openai.com
#https://platform.openai.com/docs/api-reference

In [ ]:
%load_ext jupyter_ai_magics

In [ ]:
%load_ext jupyter_ai

In [ ]:
!pip install JupyterLab

In [ ]:
pip install openai

In [ ]:
%%ai chatgpt --format code

Please generate a code in python to solve the 2D Laplace equation in cartesian cordinates. 
Solve the equation on the square domain x=(0,1) and y =(0,1) with vanishing boundary conditions.
Plot the solution using Matplotlib.
#Please also provide an explaination.


In [ ]:
%%ai chatgpt --format code

Please generate a code in python to solve the 2D Laplace equation in cartesian cordinates. 
Solve the equation on the square domain x=(0,1) and y =(0,1) with vanishing boundary conditions.
Plot the solution using Matplotlib.
#Please also provide an explaination.


In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

In [ ]:
import openai

MAX_RETRIES = 3  # Maximum number of retries

def call_openai_api(prompt, max_tokens=100, temperature=0.7):
  for _ in range(MAX_RETRIES):
    try:
      response = openai.Completion.create(
          engine="text-davinci-003",
          prompt=prompt,
          max_tokens=max_tokens,
          temperature=temperature
      )
      return response
    except openai.error.RateLimitError as e:
      print(f"API Rate Limit Hit! Retrying in {2**_} seconds...")
      time.sleep(2**_)  # Exponential backoff with increasing wait time
    except Exception as e:
      print(f"An error occurred: {e}")
      raise  # Re-raise the exception for other errors

# Example usage
prompt = "Write a story about a robot who falls in love with a human."
response = call_openai_api(prompt)
print(response.choices[0].text)


In [ ]:
pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define domain size and grid points
Lx = 1.0  # Length in x-direction
Ly = 1.0  # Length in y-direction
Nx = 101  # Number of grid points in x-direction (including boundaries)
Ny = 101  # Number of grid points in y-direction (including boundaries)

# Create meshgrid for x and y coordinates
dx = Lx / (Nx - 1)  # Grid spacing in x
dy = Ly / (Ny - 1)  # Grid spacing in y
x, y = np.meshgrid(np.linspace(0, Lx, Nx), np.linspace(0, Ly, Ny))

# Define the Laplace operator (stencil for 5-point finite difference)
laplacian = np.zeros((Ny, Nx))
laplacian[1:-1, 1:-1] = (
    -2.0 * (laplacian[1:-1, 1:-1] + laplacian[1:-1, 0:-2] + laplacian[0:-2, 1:-1])
    / (dx**2)
    + 2.0 * (laplacian[1:-1, 2:] + laplacian[2:, 1:-1]) / (dx**2)
    - 2.0 * (laplacian[1:-1, 1:-1] + laplacian[0:-2, 1:-1]) / (dy**2)
    + 2.0 * (laplacian[2:, 1:-1] + laplacian[1:-1, 2:]) / (dy**2)
)

# Set up vanishing boundary conditions (solution is zero on all edges)
laplacian[0, :] = 0.0  # Set bottom edge to zero
laplacian[:, 0] = 0.0  # Set left edge to zero
laplacian[Ny - 1, :] = 0.0  # Set top edge to zero
laplacian[:, Nx - 1] = 0.0  # Set right edge to zero

# Solve the Laplace equation using a relaxation method (iterative)
# (This is a simple example, more efficient methods like conjugate gradients exist)
tolerance = 1e-6
max_iterations = 1000
phi = np.zeros((Ny, Nx))  # Initialize potential

for _ in range(max_iterations):
    phi_old = phi.copy()
    phi[1:-1, 1:-1] = (
        laplacian[1:-1, 1:-1] + phi_old[0:-2, 1:-1] + phi_old[1:-1, 0:-2]
        + phi_old[2:, 1:-1] + phi_old[1:-1, 2:]
    ) / 4.0

    # Check convergence criteria (change in solution less than tolerance)
    if np.abs(phi - phi_old).max() < tolerance:
        break

# Plot the solution
plt.contourf(x, y, phi, cmap="coolwarm")
plt.colorbar(label="Potential")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Solution of the 2D Laplace Equation with Vanishing Boundary Conditions")
plt.show()


In [10]:
import requests
!pip install beautifulsoup4
!pip install pandas
from bs4 import BeautifulSoup
import pandas as pd

def scrape_weather_data():
    # Step 1: Web scraping to find the correct file
    url = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/2021/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the link corresponding to the specified timestamp
    target_timestamp = '2022-02-07 14:03'
    file_link = None
    for link in soup.find_all('a', href=True):
        if target_timestamp in link.text:
            file_link = link['href']
            break

    if not file_link:
        print(f"File not found for timestamp {target_timestamp}.")
        exit()

    # Step 2: Download the file
    base_url = 'https://www.ncei.noaa.gov'
    full_url = base_url + file_link
    filename = file_link.split('/')[-1]

    print(f"Downloading file: {filename} ...")
    response = requests.get(full_url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded successfully: {filename}")

    return filename

def analyze_weather_data(filename):
    # Step 3: Load data into Pandas
    df = pd.read_csv(filename)

    # Find record(s) with the highest HourlyDryBulbTemperature
    max_temp = df['HourlyDryBulbTemperature'].max()
    max_temp_records = df[df['HourlyDryBulbTemperature'] == max_temp]

    print(f"Records with highest HourlyDryBulbTemperature ({max_temp}°F):")
    print(max_temp_records)

if __name__ == "__main__":
    downloaded_file = scrape_weather_data()
    analyze_weather_data(downloaded_file)


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\_markupbase.py:36: RuntimeWarning: coroutine 'download_file_async' was never awaited
  def getpos(self):
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\_markupbase.py:36: RuntimeWarning: coroutine 'main_async' was never awaited
  def getpos(self):


File not found for timestamp 2022-02-07 14:03.


TypeError: can only concatenate str (not "NoneType") to str

SyntaxError: expected 'except' or 'finally' block (628828753.py, line 47)

In [2]:
import requests
import os
import aiohttp
import asyncio
import zipfile
from concurrent.futures import ThreadPoolExecutor

# Define the list of file URLs
download_uris = [
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2018_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2020_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2220_Q1.zip",
]

# Create the 'downloads' directory if it doesn't exist
if not os.path.exists("downloads"):
    os.makedirs("downloads")

# Step 1: Download files asynchronously using aiohttp
async def download_file_async(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            filename = os.path.basename(url)
            zip_path = os.path.join("downloads", filename)
            with open(zip_path, "wb") as f:
                f.write(await response.read())
            return zip_path

async def main_async():
    tasks = [download_file_async(url) for url in download_uris]
    await asyncio.gather(*tasks)

# Step 3: Download using ThreadPoolExecutor for additional concurrency
def download_files_threadpool():
    with ThreadPoolExecutor(max_workers=len(download_uris)) as executor:
        zip_paths = list(executor.map(download_file_async, download_uris))
        for zip_path in zip_paths:
            extract_csv_from_zip(zip_path)
            os.remove(zip_path)  # Delete the ZIP file



In [3]:
import os

def create_downloads_directory():
    if not os.path.exists("downloads"):
        os.makedirs("downloads")

create_downloads_directory()


In [4]:
import requests

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)

download_file = [
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2018_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2020_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2220_Q1.zip",
]

# Example usage:
#download_file("https://example.com/file1.zip", "downloads/file1.zip")


In [6]:
import os

def extract_filename_from_uri(uri):
    return os.path.basename(uri)

# Example usage:
uris = [
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2018_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2020_Q1.zip",
    "https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2220_Q1.zip",
]

for uri in uris:
    filename = extract_filename_from_uri(uri)
    print(f"Original filename for {uri}: {filename}")


Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2018_Q4.zip: Divvy_Trips_2018_Q4.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip: Divvy_Trips_2019_Q1.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip: Divvy_Trips_2019_Q2.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip: Divvy_Trips_2019_Q3.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip: Divvy_Trips_2019_Q4.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2020_Q1.zip: Divvy_Trips_2020_Q1.zip
Original filename for https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2220_Q1.zip: Divvy_Trips_2220_Q1.zip


In [7]:
import os
import zipfile

def extract_csv_from_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        csv_filename = None
        for file_info in zip_ref.infolist():
            if file_info.filename.endswith(".csv"):
                csv_filename = file_info.filename
                zip_ref.extract(csv_filename, "downloads")
                break

    if csv_filename:
        os.remove(zip_filename)
        print(f"Extracted CSV: {csv_filename}")
    else:
        print(f"No CSV found in {zip_filename}")

# Example usage:
zip_filename = "downloads/file1.zip"
extract_csv_from_zip(zip_filename)


FileNotFoundError: [Errno 2] No such file or directory: 'downloads/file1.zip'